# Урок 5. Задача оттока: варианты постановки, возможные способы решения
1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности  
***

In [54]:
import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix,  f1_score, roc_auc_score, \
                            precision_score, classification_report, precision_recall_curve

In [2]:
df = pd.read_csv("materials/churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
X = df[['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
        'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
        'IsActiveMember', 'EstimatedSalary']]
y = df['Exited']

In [4]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [5]:
# делим названия колонок данных на категориальные/численные
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

## Создаем Pipelines

### Random Forest

In [6]:
# делаем дамми переменные, только не через пандас, а через OHE из sklearn
ohe = OneHotEncoder(sparse=False)
# создаем обьект с моделью случайного леса
rfc = RandomForestClassifier(random_state=42)
# создаем обьект робаст скейлера
rs = RobustScaler()

# создаем column_transformer к категориальным колонкам применяем OHE, к числовым - RobustScaler
ct = make_column_transformer((ohe, categorical_columns),
                             (rs, continuous_columns))

# создаем пайплайн (он создаст дамми переменные по категориальным колонкам и получившиеся данные передаст в случайный лес)
pipe = make_pipeline(ct, rfc)
# подаем данные в пайплайн
# pipe.fit(X_train, y_train)

#### Подбираем параметры для модели

In [7]:
params={'randomforestclassifier__n_estimators':[50, 100, 200],
        'randomforestclassifier__min_samples_leaf':[1, 2, 3, 5, 10, 20],
        'randomforestclassifier__max_depth':[1, 2, 3, 4, 5, 6]
        }

grid = GridSearchCV(pipe,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X, y)
search.best_params_

{'randomforestclassifier__max_depth': 6,
 'randomforestclassifier__min_samples_leaf': 3,
 'randomforestclassifier__n_estimators': 100}

#### Проходим кросс валидацию, используя F1-score

In [8]:
rfc_best = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=6, min_samples_leaf=3)
pipe_best = make_pipeline(ct, rfc)

cv_score = cross_val_score(pipe_best, X, y, cv=6, scoring='f1_weighted')

In [9]:
cv_score.mean()

0.8476356013120019

### Gradient Boosting

In [10]:
ohe = OneHotEncoder(sparse=False)
gbc = GradientBoostingClassifier(random_state=42)

ct = make_column_transformer((ohe, categorical_columns),
                             (rs, continuous_columns))

pipe = make_pipeline(ct, gbc)
# pipe.fit(X_train, y_train)

In [16]:
params={'gradientboostingclassifier__learning_rate':[0.1, 0.075, 0.05, 0.025, 0.01],
        'gradientboostingclassifier__n_estimators':[50, 100, 200],
        'gradientboostingclassifier__min_samples_leaf':[1, 3, 5, 10],
        'gradientboostingclassifier__max_depth':[1, 2, 3, 4]
        }

grid = GridSearchCV(pipe,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'gradientboostingclassifier__learning_rate': 0.075,
 'gradientboostingclassifier__max_depth': 3,
 'gradientboostingclassifier__min_samples_leaf': 3,
 'gradientboostingclassifier__n_estimators': 200}

In [17]:
gbc_best = GradientBoostingClassifier(random_state=42, learning_rate=0.075, n_estimators=200, max_depth=3, min_samples_leaf=3)
pipe_best = make_pipeline(ct, gbc_best)

cv_score = cross_val_score(pipe_best, X, y, cv=6, scoring='f1_weighted')

In [18]:
cv_score.mean()

0.850723768061036

### Logistic Regression

In [19]:
lr = LogisticRegression(random_state=42)

pipe = make_pipeline(ct, lr)

# pipe.fit(X_train, y_train)

In [39]:
params={'logisticregression__C':list(map(lambda x: round(x, 2), np.linspace(0.1, 1.1, 10))),
        }

grid = GridSearchCV(pipe,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X, y)
search.best_params_

{'logisticregression__C': 0.32}

In [40]:
lr_best = LogisticRegression(random_state=42, C=0.32)
pipe_best = make_pipeline(ct, lr_best)

cv_score = cross_val_score(pipe_best, X, y, cv=6, scoring='f1_weighted')

In [41]:
cv_score.mean()

0.7733018969827511

## Оценка экономической модели, использую GradientBoosting

In [42]:
pipe_best = make_pipeline(ct, gbc_best)
pipe_best.fit(X_train, y_train)

y_pred = pipe_best.predict_proba(X_test)
y_pred = y_pred[:, 1]

In [57]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], fscore[ix], precision[ix], recall[ix]))

Best Threshold=0.375024, F-Score=0.647, Precision=0.671, Recall=0.625


In [55]:
precision

array([0.20425361, 0.20393416, 0.20401606, ..., 1.        , 1.        ,
       1.        ])

In [56]:
recall

array([1.        , 0.99803536, 0.99803536, ..., 0.00392927, 0.00196464,
       0.        ])

In [46]:
conf_matrix = confusion_matrix(y_test, y_pred>thresholds[ix])
conf_matrix

array([[1835,  156],
       [ 192,  317]], dtype=int64)

### Интерпретация матрицы ошибок:
1835 клиентов были обозначены как стабильные (не собираются уходить) И эта гипотеза оказалась верной  
156 клиентов были обозначены стабильными. Но на самом деле ушли  
192 клиента были обозначены как собирающиеся уходить. Но на самом деле не собирались уходить  
317 клиентов были обозначены как собирающиеся уходить. И эта гипотеза оказалась верной  
  
#### Экономические расчеты:  
  
На 1835 клиентов мы не потратили ничего и ничего не потеряли  
156 клиентов у нас ушло в отток, так как мы думали, что они стабильные, и ошиблись  
192 клиента мы просто так постарались привлечь, хотя этого не требовалось  
317 клиентов были вовремя "привлечены" и не ушли в отток  
  
192 + 317 = 509 - долларов потрачено на удержание клиентов  
317 * 2 = 634 - доллара доход с клиентов, которые не ушли в отток  
неизвестно, сколько денег было потеряно, не привлеча 156 клиентов, ушедших в отток  
  
Итого, модель работает минимально прибыльно. Максимизируя метрику полноты (recall) мы будем меньше тратить денег впустую, привлекая и без того стабильных клиентов. Однако при этом, вполне может уменьшится количество правильно опознанных клиентов, собирающихся уйти в отток

***
P.S. Подбор гиперпараметров модели был произведен сразу же, на этапе выбора модели. Сбалансированый F-score после подбора параметров увеличился с 0.7 до 0.85. Не очевидно, насколько большой прирост это дало в пересчете на финансы, но он явно есть :)